# LIBRARIES

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
from torch import optim
import torchmetrics

import numpy as np

In [2]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: markgich (use `wandb login --relogin` to force relogin)


True

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# DATASET

In [4]:

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)


# LINEAR NEURAL NET

In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Dropout(p=0.25),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(p=0.25),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()
model.to(device)
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.25, inplace=False)
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.25, inplace=False)
    (6): Linear(in_features=512, out_features=10, bias=True)
    (7): ReLU()
  )
)

#  CONVOLUTION NEURAL NETWORK

class ConvNet(nn.Module):
    def __init__(self, input_shape=(1,28,28)):
        super(ConvNet, self).__init__()

        # DEFINE THE CONVOLUTION LAYERS

        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 64,3)
        self.conv3 = nn.Conv2d(64, 128, 3)

        # DEFINE THE POOLING LATER

        self.pool = nn.MaxPool2d(2, 2)

        n_size = self._get_conv_output(input_shape)

        # DEFINE THE LINEAR CLASSS

        self.fc1 = nn.Linear(n_size, 512)
        self.fc2 = nn.Linear(512, 10)

        # DEFINE THE DROPOUT
        self.dropout = nn.Dropout(p=0.25)

    def _get_conv_output(self, shape):
        batch_size = 1
        input = torch.autograd.Variable(torch.rand(batch_size, *shape))
        output_feat = self._forward_features(input)
        n_size = output_feat.data.view(batch_size, -1).size(1)
        return n_size

    def _forward_features(self, X):
        X = self.pool(F.relu(self.conv1(X)))
        X = self.pool(F.relu(self.conv2(X)))
        X = self.pool(F.relu(self.conv3(X)))
        return X

    def forward(self, X):
        X = self._forward_features(X)
        X = X.view(X.size(0), -1)
        X = self.dropout(X) # DROPOUT
        X = F.relu(self.fc1(X))
        X = self.dropout(X) # DROPOUT
        X = self.fc2(X)
        return X
        
model = ConvNet()

# PUSH MODEL TO DEVICE(CUDA)
model.to(device)
print(model)

# HYPERPARAMETERS

In [6]:
config = dict(learning_rate = 1e-3,
                batch_size = 64,
                epochs = 10,
                loss = nn.NLLLoss(),
                )

In [7]:
for k, v in config.items():
    print(f"{k}: {v}")

learning_rate: 0.001
batch_size: 64
epochs: 10


# TRAINING

In [8]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    #acc = 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        acc = torchmetrics.functional.accuracy(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"train loss: {loss:>3f}  [{current:>5d}/{size:>5d}]")
            #wandb.log({"train loss": loss, "train accuracy": acc})
    print(f"Train Metrics: \n Train Accuracy: {(100*acc):>0.1f}%, Train Loss: {loss:>8f} \n")
    wandb.log({"Train Accuracy": acc, "Train Loss": loss })


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            acc = torchmetrics.functional.accuracy(pred, y).item()
            #correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    wandb.log({"Test Accuracy":acc, "Test Loss": test_loss})

    test_loss /= num_batches
    correct /= size
    #print(f"Test Metrics: \n Test Accuracy: {(100*correct):>0.1f}%, Test Loss: {test_loss:>8f} \n")
    print(f"Test Metrics: \n Test Accuracy: {(100*acc):>0.1f}%, Test Loss: {test_loss:>8f} \n")
    #wandb.log({"Test Accuracy":acc, "Test Loss": test_loss})

In [9]:
# define loss and optimizer

#loss_fn = nn.NLLLoss()
loss_fn = config.get("loss")
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adagrad(model.parameters(), lr=config.get("learning_rate"))


In [10]:
%%time

with wandb.init(project="demo_wandb_fashionmnsit_test", config=config):
    wandb.watch(model, criterion=loss_fn, log="all", log_freq=10)
    epochs = 10
    for t in range(epochs):

        print(f"Epoch {t+1}\n-------------------------------")
        train_loop(train_dataloader, model, loss_fn, optimizer)
        test_loop(test_dataloader, model, loss_fn)
    wandb.save(torch.save(model.state_dict(), f=f"./models/fashion_mnist_{t}.pt"))
print("Done!")


Epoch 1
-------------------------------
train loss: 2.306896  [    0/60000]
train loss: 1.295077  [ 6400/60000]
train loss: 1.046859  [12800/60000]
train loss: 1.190677  [19200/60000]
train loss: 1.211662  [25600/60000]
train loss: 1.217390  [32000/60000]
train loss: 0.919782  [38400/60000]
train loss: 1.175587  [44800/60000]
train loss: 1.001915  [51200/60000]
train loss: 1.105289  [57600/60000]
Train Metrics: 
 Train Accuracy: 65.6%, Train Loss: 1.194652 

Test Metrics: 
 Test Accuracy: 75.0%, Test Loss: 1.008298 

Epoch 2
-------------------------------
train loss: 0.871998  [    0/60000]
train loss: 0.976365  [ 6400/60000]
train loss: 0.839523  [12800/60000]
train loss: 1.041528  [19200/60000]
train loss: 1.043180  [25600/60000]
train loss: 1.157210  [32000/60000]
train loss: 0.805272  [38400/60000]
train loss: 1.095513  [44800/60000]
train loss: 0.954295  [51200/60000]
train loss: 1.027219  [57600/60000]
Train Metrics: 
 Train Accuracy: 62.5%, Train Loss: 1.142550 

Test Metrics: 

wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Test Metrics: 
 Test Accuracy: 68.8%, Test Loss: 0.871815 



Train Accuracy,0.625
Train Loss,0.9585
_runtime,133
_timestamp,1625922504
_step,19
Test Accuracy,0.6875
Test Loss,136.87503


Train Accuracy,█▁▁▁▁▁▁▁█▁
Train Loss,█▇▆▃▄▄▄▁▄▂
_runtime,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇██
_timestamp,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇██
_step,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
Test Accuracy,▅▁▁▁▅▅▅▁█▁
Test Loss,█▅▄▃▃▂▂▁▁▁


Done!
Wall time: 2min 18s
